In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

from unicodedata import category
from tqdm import tqdm
#from tqdm import tqdm_notebook as tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

import WOSutilities as wosutil

path2rawdata = '/mnt/disks/vault/wos2017-parsed/'

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
# we need to load articles and categories
article_df = wosutil.load_wos_data(name = 'article', 
                                   path2rawdata = path2rawdata,
                                   year_list = None, 
                                   columns = ['ArticleID', 'PubYear', 'Doctypes', 'Title','Journal', 'Journal Abbreviation ISO'], 
                                   dropna = ['ArticleID', 'PubYear','Title'], 
                                   duplicate_subset = ['ArticleID'],
                                   isindict = {'Doctypes':np.sort(['Article','Letter','Review','Note'])}, 
                                   verbose = 50)

# select all categories for all the aricles published
category_list = np.array(['Physics, Applied'])  

category_df = wosutil.load_wos_data(name = 'category', 
                                    path2rawdata = path2rawdata,
                                    year_list = None,
                                    columns = ['ArticleID', 'Type', 'Label'], 
                                    dropna = ['ArticleID'], 
                                    duplicate_subset = None,
                                    isindict = {'Label':category_list},
                                    verbose = 100)

50
100
150
200
250
300
350
400
450
Final DF Shape (39171585, 6)
100
200
300
400
Final DF Shape (1289055, 3)


In [12]:
# How many article ids are present in category_df
print(len(category_df['ArticleID'].unique()))

# Are there NaN values in article id
print(category_df.isnull().values.any())

# Are there NaN values in article id
print(set(category_df['ArticleID'].isna()))

# Are there NaN values in Label
print(set(category_df['Label'].isna()))

1289055
False
{False}
{False}


In [17]:
# what is the distribution of article ids for category types
category_df.groupby(['Label']).size()

Label
Physics, Applied    1289055
dtype: int64

In [13]:
# Checking articles dataframe
print(len(article_df['ArticleID'].unique()))

# Are there NaN values in article id
print(article_df.isnull().values.any())

39171585
False


In [14]:
# joining citations with articles and categories

# joining the dataframes
data = None
data = category_df.merge(article_df, 
                         how = 'inner',
                         on = 'ArticleID')

NameError: name 'date' is not defined

In [16]:
data.count()

ArticleID                   1031701
Type                        1031701
Label                       1031701
PubYear                     1031701
Doctypes                    1031701
Title                       1031701
Journal                     1031701
Journal Abbreviation ISO    1031701
dtype: int64

In [19]:
data = data.drop_duplicates()
data.to_pickle("/mnt/disks/vault/analysis-data/applied_physics_article_categories/articles_and_categories_applied_physics.pql")